In [1]:
import nest_asyncio
nest_asyncio.apply()
del nest_asyncio

In [2]:
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)

Building: found in cache, done.
Messages from stanc:
Warning in '/var/folders/79/lndg0b5n2q3b40m_ty2kxlm40000gn/T/httpstan_f6j1s3o1/model_qv763guo.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/var/folders/79/lndg0b5n2q3b40m_ty2kxlm40000gn/T/httpstan_f6j1s3o1/model_qv763guo.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [3]:
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas
print(df.describe().T)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 0.000496 seconds
  1000 transitions using 10 leapfrog steps per transition would take 4.96 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.14 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!


                count       mean       std           min        25%  \
parameters                                                            
lp__           4000.0 -39.509640  2.646235 -5.072949e+01 -41.117389   
accept_stat__  4000.0   0.868156  0.205266  9.810667e-27   0.854323   
stepsize__     4000.0   0.309425  0.018318  2.827443e-01   0.299285   
treedepth__    4000.0   3.618250  0.516317  1.000000e+00   3.000000   
n_leapfrog__   4000.0  13.200000  3.656685  1.000000e+00  15.000000   
divergent__    4000.0   0.000250  0.015811  0.000000e+00   0.000000   
energy__       4000.0  44.485173  3.489856  3.558989e+01  42.081159   
mu             4000.0   7.924777  5.177116 -8.647502e+00   4.543346   
tau            4000.0   6.809661  5.785845  2.994943e-03   2.574551   
eta.1          4000.0   0.394421  0.910154 -3.125239e+00  -0.186816   
eta.2          4000.0   0.029205  0.840400 -2.714981e+00  -0.527592   
eta.3          4000.0  -0.221261  0.941423 -3.608201e+00  -0.845483   
eta.4 